<a href="https://colab.research.google.com/github/sreeduttsasikumar/Coursera_Capstone/blob/main/Capstone_Project_Battle_of_Neighbourhoods.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# <center>Capstone Project - The Battle of the Neighborhoods</center>
# <center> Analyzing major cities in Kerala, India for choosing the site for a Residential Project <center>

## Table of contents
* [Introduction: Business Problem](#introduction)
* [Data](#data)

## Introduction: Business Problem <a name="introduction"></a>

One of the major Real Estate investor in India, based out of Mumbai, **wants to construct a 5-star Residential Complex in any major city in Kerala**. To that purpose, he wants to analyze the neighbourhood of the cities and choose a neighbourhood/city which will be suitable for the project.

He is planning to have a **water facing villa theme**, and the water body can be anything like Beaches, Lakes, River, Backwater, etc. Also, the **locality should be suitable for a 5 star residential villa with all necessary facilities and ameninites nearby**

We will use the power of Data Science to map the requirements described against the data available and find out one/mutiple cities which will be well suited for the project.

## Data <a name="data"></a>

Based on definition of our problem, factors that will influence our decission are:
* presence of water body in locality
* presense of amentities for household such as Grocery, Schools, Hospitals, etc
* should not be a trending/happening locality as that won't be suitable for a residential area

We decided to use the wikipedia page which lists the major Muncipality and Muncipality Corporations in Kerala. Will extract the data from the table and will use those locations for finding out the perfect location as described in the requirement.

By using the locations details downloaded, corresponding latitudes and longitudes will be captured using geocoder api. Also using foursquare api we will find out the neighbouring venues of those locations which will be the basis for the analysis we do in this project

Lets start the journey of data exploration, data enriching, data cleaning, data transfomration, etc etc

Use below cell to consolidate all the library package importing

In [74]:
#pandas don't need much info. Anyways imported for dataframe functionality
import pandas as pd
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)
# convert an address into latitude and longitude values
from geopy.geocoders import Nominatim
# Matplotlib and associated plotting modules
import matplotlib.cm as cm
import matplotlib.colors as colors
# map rendering library
import folium
# library to handle JSON files 
import json
# library to handle requests
import requests

Lets download the data containing the major cities in Kerala from the identified wikipedia page

In [75]:
#Assign the wikipedia url to variable url
url = 'https://en.wikipedia.org/wiki/List_of_cities_and_towns_in_Kerala'
#fetch from the url using panda function
data_from_url = pd.read_html(url)
#get the second table which is the relevant table for our project
kerala_city_df = data_from_url[1]
#drop columns not needed for analysis
kerala_city_df.drop(kerala_city_df.columns[[0, 2, 3, 4]], axis = 1, inplace = True) 
#rename columns to meaningful names
kerala_city_df.columns = ['City', 'Muncipality']
#correct a few data for proper address resolution and coordinates fecthing by geocoder
kerala_city_df.City[kerala_city_df.City=='Thiruvananthapuram'] = 'Trivandrum, TVM'
kerala_city_df.Muncipality[kerala_city_df.Muncipality=='Thiruvananthapuram'] = 'Trivandrum'
kerala_city_df.drop(index = kerala_city_df[kerala_city_df['City']=='Municipalities'].index[0], inplace= True)
kerala_city_df.reset_index(inplace= True, drop=True)
print("Total number of cities available for analysis are {}.".format(kerala_city_df.shape[0]))
kerala_city_df.head()

Total number of cities available for analysis are 93.


,City,Muncipality
0,"Trivandrum, TVM",Trivandrum
1,Kozhikode,Kozhikode
2,Kochi,Ernakulam
3,Kollam,Kollam
4,Thrissur,Thrissur


####Lets use Geocoder to fetch the latitude and longitudes of the cities and update them in the dataframe

Let's define below function to convert the city to corresponding latitudes and longitudes

In [76]:
 def lat_lon_from_address(cities):  
  city_lat_long_temp = []
  for city in cities: 
    geolocator = Nominatim(user_agent="city_explorer")
    location = geolocator.geocode(city)
    latitude = location.latitude
    longitude = location.longitude
    #print('The geograpical coordinate of {} are {}, {}.'.format(city, latitude, longitude))
    city_lat_long_temp.append([city, latitude, longitude])
  return (city_lat_long_temp)

Call the above defined function by passing the city names as parameters and assign the newly found latitudes and longitudes in a new dataframe. This will be then merged with main dataframe kerala_city_df

In [77]:
#call the function
city_lat_long = pd.DataFrame(lat_lon_from_address(kerala_city_df['City']))
#set columns for the new dataframe
city_lat_long.columns = ['City', 'City Latitude', 'City Longitude']
#using pandas merge function merge the initial data set and the coordinte data set using the common column City
kerala_city_df = pd.merge(kerala_city_df, city_lat_long, left_on='City', right_on='City')
kerala_city_df.head()

,City,Muncipality,City Latitude,City Longitude
0,"Trivandrum, TVM",Trivandrum,8.487803,76.951847
1,Kozhikode,Kozhikode,11.258608,75.778874
2,Kochi,Ernakulam,9.931308,76.267414
3,Kollam,Kollam,8.887951,76.595501
4,Thrissur,Thrissur,10.525626,76.213254


#### Create a map of Kerala with neighborhoods superimposed on top.

In [78]:
#get geo location of Kerala on which the cities can be super imposed
geolocator = Nominatim(user_agent="city_explorer")
location = geolocator.geocode('Kerala, KL')
latitude = location.latitude
longitude = location.longitude
print('The geograpical coordinate of Kerala are {}, {}.'.format(latitude, longitude))

The geograpical coordinate of Kerala are 9.5797046, 76.5691745.


In [79]:
map_kerala = folium.Map(location=[latitude, longitude], zoom_start=10)

# add markers to map
for lat, lng, muncipality, city in zip(kerala_city_df['City Latitude'], kerala_city_df['City Longitude'], kerala_city_df['Muncipality'], kerala_city_df['City']):
    label = '{}, {}'.format(city, muncipality)
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color='blue',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7,
        parse_html=False).add_to(map_kerala)  
    
map_kerala

#### Define Foursquare Credentials and Version

In [80]:
CLIENT_ID = 'UDO03A1SPAJOCS4SJODJ31ORPYBC2NA2M2ND4QLRL51LNO3H' # your Foursquare ID
CLIENT_SECRET = '3LBAEVCJ21YGCIMYN1LCZBXWCHA4CE1QI511MI4HJP4ASSXN' # your Foursquare Secret
VERSION = '20180605' # Foursquare API version
LIMIT = 100 # A default Foursquare API limit value

#### Let's create a function to fetch nearby venues around the cities in Kerala

In [83]:
def getNearbyVenues(names, muncipalities, latitudes, longitudes, radius=1000):
    
    venues_list=[]
    for name, muncipality, lat, lng in zip(names, muncipalities, latitudes, longitudes):
        #print(name)
            
        # create the API request URL
        url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
            CLIENT_ID, 
            CLIENT_SECRET, 
            VERSION, 
            lat, 
            lng, 
            radius, 
            LIMIT)
            
        # make the GET request
        results = requests.get(url).json()["response"]['groups'][0]['items']
        
        # return only relevant information for each nearby venue
        venues_list.append([(
            name, 
            lat, 
            lng, 
            muncipality,
            v['venue']['name'], 
            v['venue']['location']['lat'], 
            v['venue']['location']['lng'],  
            v['venue']['categories'][0]['name']) for v in results])

    nearby_venues = pd.DataFrame([item for venue_list in venues_list for item in venue_list])
    nearby_venues.columns = ['City', 
                  'City Latitude', 
                  'City Longitude', 
                  'Muncipality', 
                  'Venue', 
                  'Venue Latitude', 
                  'Venue Longitude', 
                  'Venue Category']
    
    return(nearby_venues)

#### Now write the code to run the above function on each neighborhood and create a new dataframe called _kerala_venues_.

In [84]:
kerala_venues = getNearbyVenues(names=kerala_city_df['City'],
                                   muncipalities=kerala_city_df['Muncipality'],
                                   latitudes=kerala_city_df['City Latitude'],
                                   longitudes=kerala_city_df['City Longitude']
                                  )
kerala_venues.head()

,City,City Latitude,City Longitude,Muncipality,Venue,Venue Latitude,Venue Longitude,Venue Category
0,"Trivandrum, TVM",8.487803,76.951847,Trivandrum,Nila Theater,8.489039,76.953007,Movie Theater
1,"Trivandrum, TVM",8.487803,76.951847,Trivandrum,Kairali Sree Nila Theatre Complex,8.489033,76.953024,Multiplex
2,"Trivandrum, TVM",8.487803,76.951847,Trivandrum,Ariesplex SL Cinemas,8.488652,76.946331,Multiplex
3,"Trivandrum, TVM",8.487803,76.951847,Trivandrum,Hotel High Land,8.488651,76.949176,Arcade
4,"Trivandrum, TVM",8.487803,76.951847,Trivandrum,Appolo Dimora,8.488882,76.950888,Hotel


#### Let's check the size of the resulting dataframe

In [87]:
print("Total {} venues were identified amongst {} cities from Kerala".format(kerala_venues.shape[0], kerala_city_df.shape[0]))

Total 459 venues were identified amongst 93 cities from Kerala


Let's check how many venues were returned for each City

In [93]:
kerala_venues.groupby('City').count()

,City Latitude,City Longitude,Muncipality,Venue,Venue Latitude,Venue Longitude,Venue Category
City,,,,,,,
Adoor,2,2,2,2,2,2,2
Aluva,4,4,4,4,4,4,4
Angamaly,5,5,5,5,5,5,5
Anthoor,4,4,4,4,4,4,4
Attingal,8,8,8,8,8,8,8
Chalakudy,4,4,4,4,4,4,4
Cherpulassery,4,4,4,4,4,4,4
Cherthala,6,6,6,6,6,6,6
Chittur-Thathamangalam,1,1,1,1,1,1,1


#### Let's find out how many unique categories can be curated from all the returned venues

In [94]:
print('There are {} uniques categories.'.format(len(kerala_venues['Venue Category'].unique())))

There are 97 uniques categories.


Analyze each neighbourhood

In [95]:
# one hot encoding
kerala_onehot = pd.get_dummies(kerala_venues[['Venue Category']], prefix="", prefix_sep="")

# add neighborhood column back to dataframe
kerala_onehot['City'] = kerala_venues['City'] 

# move neighborhood column to the first column
fixed_columns = [kerala_onehot.columns[-1]] + list(kerala_onehot.columns[:-1])
kerala_onehot = kerala_onehot[fixed_columns]

kerala_onehot.head()

,City,ATM,Accessories Store,Airport,Airport Terminal,Arcade,Arts & Crafts Store,Asian Restaurant,Auditorium,Bakery,Bar,Beach,Bed & Breakfast,Boat or Ferry,Bookstore,Boutique,Bowling Alley,Bridal Shop,Burger Joint,Bus Line,Bus Station,Cafeteria,Café,Campground,Chinese Restaurant,Clothing Store,Coffee Shop,Comfort Food Restaurant,Concert Hall,Convenience Store,Currency Exchange,Department Store,Diner,Electronics Store,Falafel Restaurant,Fast Food Restaurant,Fish Market,Flea Market,Food,Food & Drink Shop,Food Court,Football Stadium,Forest,Fried Chicken Joint,Furniture / Home Store,General Travel,Gift Shop,Grocery Store,Gym,Hotel,IT Services,Ice Cream Shop,Indian Restaurant,Indie Movie Theater,Jewelry Store,Juice Bar,Kerala Restaurant,Lake,Light Rail Station,Liquor Store,Market,Mattress Store,Men's Store,Middle Eastern Restaurant,Mobile Phone Shop,Motel,Movie Theater,Multicuisine Indian Restaurant,Multiplex,Other Great Outdoors,Outdoor Supply Store,Paper / Office Supplies Store,Park,Pharmacy,Pizza Place,Platform,Playground,Pub,Resort,Restaurant,River,Scenic Lookout,Shopping Mall,Snack Place,Southern / Soul Food Restaurant,Spa,Sporting Goods Shop,Sports Club,Stadium,Student Center,Supermarket,Temple,Track,Track Stadium,Train Station,Travel & Transport,Vegetarian / Vegan Restaurant,Women's Store
0,"Trivandrum, TVM",0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
1,"Trivandrum, TVM",0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
2,"Trivandrum, TVM",0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
3,"Trivandrum, TVM",0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
4,"Trivandrum, TVM",0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0


#### Next, let's group rows by city and by taking the mean of the frequency of occurrence of each category

In [96]:
kerala_grouped = kerala_onehot.groupby('City').mean().reset_index()
kerala_grouped

,City,ATM,Accessories Store,Airport,Airport Terminal,Arcade,Arts & Crafts Store,Asian Restaurant,Auditorium,Bakery,Bar,Beach,Bed & Breakfast,Boat or Ferry,Bookstore,Boutique,Bowling Alley,Bridal Shop,Burger Joint,Bus Line,Bus Station,Cafeteria,Café,Campground,Chinese Restaurant,Clothing Store,Coffee Shop,Comfort Food Restaurant,Concert Hall,Convenience Store,Currency Exchange,Department Store,Diner,Electronics Store,Falafel Restaurant,Fast Food Restaurant,Fish Market,Flea Market,Food,Food & Drink Shop,Food Court,Football Stadium,Forest,Fried Chicken Joint,Furniture / Home Store,General Travel,Gift Shop,Grocery Store,Gym,Hotel,IT Services,Ice Cream Shop,Indian Restaurant,Indie Movie Theater,Jewelry Store,Juice Bar,Kerala Restaurant,Lake,Light Rail Station,Liquor Store,Market,Mattress Store,Men's Store,Middle Eastern Restaurant,Mobile Phone Shop,Motel,Movie Theater,Multicuisine Indian Restaurant,Multiplex,Other Great Outdoors,Outdoor Supply Store,Paper / Office Supplies Store,Park,Pharmacy,Pizza Place,Platform,Playground,Pub,Resort,Restaurant,River,Scenic Lookout,Shopping Mall,Snack Place,Southern / Soul Food Restaurant,Spa,Sporting Goods Shop,Sports Club,Stadium,Student Center,Supermarket,Temple,Track,Track Stadium,Train Station,Travel & Transport,Vegetarian / Vegan Restaurant,Women's Store
0,Adoor,0.000000,0.000000,0.000000,0.00,0.000000,0.000000,0.000000,0.000,0.000000,0.000,0.000000,0.00,0.000000,0.000000,0.000000,0.000000,0.000,0.000000,0.000000,0.000000,0.0,0.000000,0.00,0.500000,0.000000,0.000000,0.000000,0.5,0.000000,0.000000,0.000000,0.000000,0.0,0.000000,0.000000,0.0,0.000000,0.000000,0.000000,0.000000,0.00,0.000000,0.000000,0.000000,0.000000,0.000,0.00,0.000000,0.000000,0.00,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.00,0.000000,0.000000,0.000000,0.00,0.000000,0.00,0.000000,0.000000,0.000000,0.000000,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.000000,0.000000,0.000000,0.000,0.000000,0.000000,0.000000,0.00,0.000000,0.000000,0.0,0.000000,0.000000,0.00,0.000000,0.000000,0.000000,0.000000,0.0,0.000000,0.000000
1,Aluva,0.000000,0.000000,0.000000,0.00,0.000000,0.000000,0.000000,0.000,0.000000,0.000,0.000000,0.00,0.000000,0.000000,0.000000,0.000000,0.000,0.000000,0.000000,0.000000,0.0,0.000000,0.00,0.000000,0.000000,0.000000,0.000000,0.0,0.000000,0.000000,0.000000,0.250000,0.0,0.000000,0.000000,0.0,0.000000,0.000000,0.000000,0.000000,0.00,0.000000,0.000000,0.000000,0.000000,0.000,0.00,0.000000,0.750000,0.00,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.00,0.000000,0.000000,0.000000,0.00,0.000000,0.00,0.000000,0.000000,0.000000,0.000000,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.000000,0.000000,0.000000,0.000,0.000000,0.000000,0.000000,0.00,0.000000,0.000000,0.0,0.000000,0.000000,0.00,0.000000,0.000000,0.000000,0.000000,0.0,0.000000,0.000000
2,Angamaly,0.000000,0.000000,0.000000,0.00,0.000000,0.000000,0.000000,0.000,0.000000,0.000,0.000000,0.00,0.000000,0.000000,0.000000,0.000000,0.000,0.000000,0.000000,0.200000,0.0,0.000000,0.00,0.000000,0.000000,0.000000,0.000000,0.0,0.000000,0.000000,0.000000,0.000000,0.0,0.000000,0.000000,0.0,0.000000,0.000000,0.000000,0.000000,0.00,0.000000,0.000000,0.000000,0.000000,0.000,0.00,0.000000,0.200000,0.00,0.000000,0.200000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.00,0.000000,0.000000,0.000000,0.00,0.000000,0.00,0.000000,0.000000,0.400000,0.000000,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.000000,0.000000,0.000000,0.000,0.000000,0.000000,0.000000,0.00,0.000000,0.000000,0.0,0.000000,0.000000,0.00,0.000000,0.000000,0.000000,0.000000,0.0,0.000000,0.000000
3,Anthoor,0.000000,0.000000,0.000000,0.00,0.000000,0.000000,0.000000,0.000,0.000000,0.000,0.000000,0.00,0.000000,0.000000,0.000000,0.000000,0.000,0.000000,0.000000,0.250000,0.0,0.250000,0.00,0.000000,0.000000,0.000000,0.000000,0.0,0.000000,0.000000,0.000000,0.000000,0.0,0.000000,0.000000,0.0,0.000000,0.000000,0.000000,0.000000,0.00,0.0000